In [ ]:
import os
import re
import numpy as np
import pandas as pd
import SimpleITK as sitk
import nibabel as nib
from tqdm import tqdm
from radiomics import featureextractor


In [ ]:


# ===== Path Configuration =====
converted_dir = r"D:\research\cd-ml\H\converted_nii"
roi_root = r"D:\research\cd-ml\H"
yaml_path = os.path.join(converted_dir, "tumor.yaml")
output_excel = os.path.join(converted_dir, "radiomics_features_all.xlsx")
failed_csv = os.path.join(converted_dir, "failed_cases_all.csv")


In [ ]:

# ===== Load Image (with fallback to nibabel) =====
def load_image(path):
    try:
        img = sitk.ReadImage(path)
        print(f"🟢 SimpleITK loaded: {path}")
        return img
    except:
        print(f"⚠ SimpleITK failed, trying nibabel: {path}")
        nimg = nib.load(path)
        data = nimg.get_fdata().astype(np.float32)
        simg = sitk.GetImageFromArray(data)
        spacing = [float(s) for s in nimg.header.get_zooms()[:3]]
        simg.SetSpacing(spacing)
        simg.SetOrigin((0.0, 0.0, 0.0))
        print(f"✅ Converted nibabel image to SimpleITK: {path}")
        return simg


In [ ]:

# ===== Resample image to match the mask =====
def resample_to_mask(image, reference):
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(reference)
    resampler.SetInterpolator(sitk.sitkLinear if image.GetPixelID() != sitk.sitkUInt8 else sitk.sitkNearestNeighbor)
    resampler.SetTransform(sitk.Transform())
    return resampler.Execute(image)

# ===== Initialize PyRadiomics Extractor with YAML config =====
extractor = featureextractor.RadiomicsFeatureExtractor(yaml_path)

# ===== Store results =====
ct_results, pet_results, failed_cases = [], [], []


In [ ]:

# ===== Traverse all ROI patient folders =====
for root, dirs, _ in os.walk(roi_root):
    if "ROI" not in root:
        continue
    for patient_dir in tqdm(dirs, desc="Extracting radiomics features"):
        name_clean = re.sub(r"^\d+", "", patient_dir)

        # Locate CT/PET folders
        ct_dir = next((os.path.join(converted_dir, d) for d in os.listdir(converted_dir)
                       if d.endswith("_CT") and name_clean in d), None)
        pet_dir = next((os.path.join(converted_dir, d) for d in os.listdir(converted_dir)
                        if d.endswith("_PET") and name_clean in d), None)

        # Locate mask file
        mask_file = None
        full_path = os.path.join(root, patient_dir)
        for subdir, _, files in os.walk(full_path):
            for f in files:
                if f.endswith(".nii") or f.endswith(".nii.gz"):
                    mask_file = os.path.join(subdir, f)
                    break
            if mask_file:
                break

        if not ct_dir or not pet_dir or not mask_file:
            failed_cases.append({"patient": name_clean, "reason": "Missing CT/PET/mask path"})
            continue

        try:
            ct_path = next(os.path.join(ct_dir, f) for f in os.listdir(ct_dir) if f.endswith(".nii") or f.endswith(".nii.gz"))
            pet_path = next(os.path.join(pet_dir, f) for f in os.listdir(pet_dir) if f.endswith(".nii") or f.endswith(".nii.gz"))
        except Exception as e:
            failed_cases.append({"patient": name_clean, "reason": f"Missing CT or PET file: {e}"})
            continue

        try:
            ct_img = load_image(ct_path)
            pet_img = load_image(pet_path)
            mask = load_image(mask_file)

            ct_img = resample_to_mask(ct_img, mask)
            pet_img = resample_to_mask(pet_img, mask)

            mask_arr = sitk.GetArrayViewFromImage(mask)
            labels = np.unique(mask_arr)
            labels = labels[labels != 0]
            if len(labels) == 0:
                raise ValueError("No foreground label found in mask")
            label = int(labels.max())

            ct_features = extractor.execute(ct_img, mask, label=label)
            pet_features = extractor.execute(pet_img, mask, label=label)

            ct_features["patient"] = name_clean
            pet_features["patient"] = name_clean

            ct_results.append(ct_features)
            pet_results.append(pet_features)

        except Exception as e:
            failed_cases.append({"patient": name_clean, "reason": str(e)})
            continue


In [ ]:

# ===== Save to Excel and CSV =====
df_ct = pd.DataFrame(ct_results)
df_pet = pd.DataFrame(pet_results)
with pd.ExcelWriter(output_excel) as writer:
    df_ct.to_excel(writer, sheet_name="CT", index=False)
    df_pet.to_excel(writer, sheet_name="PET", index=False)

pd.DataFrame(failed_cases).to_csv(failed_csv, index=False)

print("✅ All patients processed.")
print(f"📄 Excel output: {output_excel}")
print(f"❌ Failed cases logged in: {failed_csv}")
